In [2]:
import zipfile
import os
import pandas as pd
import numpy as np

1. Obter os dados do Kaggle

In [ ]:
zip_file_path = "c:\\Users\\rodri\\spotify-charts-all-audio-data.zip"
extract_to_path = "C:\\Users\\rodri\\OneDrive\\Documentos\\Projeto ETL"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

In [3]:

chunksize = 10**6  # Tamanho dos chunks para leitura
filepath = "C:\\Users\\rodri\\OneDrive\\Documentos\\Projeto ETL\\merged_data.csv" # Caminho para o seu arquivo

# Definir os tipos de dados esperados para cada coluna
dtypes = {
    'id': str,
    'title': str,
    'rank': str,
    'date': str,
    'artist': str,
    'url': str,
    'region': str,
    'chart': str,
    'trend': str,
    'streams': str,
    'track_id': str,
    'album': str,
    'popularity': str,
    'duration_ms': str,
    'explicit': str,
    'release_date': str,
    'available_markets': str,
    'af_danceability': str,
    'af_energy': str,
    'af_key': str,
    'af_loudness': str,
    'af_mode': str,
    'af_speechiness': str,
    'af_acousticness': str,
    'af_instrumentalness': str,
    'af_liveness': str,
    'af_valence': str,
    'af_tempo': str,
    'af_time_signature': str
}

# Lista para armazenar os dataframes filtrados
filtered_data = []

for chunk in pd.read_csv(filepath, chunksize=chunksize, dtype=dtypes, low_memory=False):

    # Converter a coluna 'date' para datetime
    chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')
    chunk['release_date'] = pd.to_datetime(chunk['release_date'], errors='coerce')
    
    # Verificar e limpar valores inválidos antes da conversão
    numeric_cols = ['rank', 'streams', 'popularity', 'duration_ms', 'explicit', 'af_danceability', 'af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_instrumentalness', 'af_liveness', 'af_valence', 'af_tempo', 'af_time_signature']

    for col in numeric_cols:
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        # Substituir NaNs por um valor padrão, se necessário
        chunk[col] = chunk[col].fillna(0)
    
    
    # Filtrar os dados para as regiões desejadas
    chunk = chunk[chunk['region'].isin(['Global', 'Brazil', 'United States', 'United Kingdom', 'Mexico', 'Germany', 'France', 'Spain', 'Bolivia', 'Chile', 'Argentina', 'Colombia'])]
    
    
    # Filtrar apenas os registros 'top200' na coluna chart
    chunk = chunk[chunk['chart'] == 'top200']
    
    # Excluir as colunas indesejadas
    columns_to_drop = ['af_acousticness', 'track_id', 'url', 'available_markets', 'af_tempo', 'af_instrumentalness']
    chunk = chunk.drop(columns=columns_to_drop)
    
    # Adicionar o chunk filtrado à lista
    filtered_data.append(chunk)
    
# Concatenar todos os chunks filtrados
filtered_df = pd.concat(filtered_data)



In [4]:
# Visualizar os primeiros dados da base filtrada

filtered_df.head()

,Unnamed: 0,title,rank,date,artist,region,chart,trend,streams,album,...,release_date,af_danceability,af_energy,af_key,af_loudness,af_mode,af_speechiness,af_liveness,af_valence,af_time_signature
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,Argentina,top200,SAME_POSITION,253019.0,El Dorado,...,2017-05-26,0.852,0.773,8.0,-2.921,0.0,0.0776,0.1590,0.907,4.0
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,Argentina,top200,MOVE_UP,223988.0,Vente Pa' Ca (feat. Maluma),...,2016-09-22,0.663,0.920,11.0,-4.070,0.0,0.2260,0.1010,0.533,4.0
2,2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,Argentina,top200,MOVE_DOWN,210943.0,Primera Cita,...,2016-08-26,0.761,0.838,4.0,-3.073,0.0,0.0502,0.1760,0.710,4.0
3,3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",Argentina,top200,SAME_POSITION,173865.0,Energía,...,2016-06-24,0.508,0.687,0.0,-4.361,1.0,0.3260,0.1260,0.555,4.0
4,4,Shaky Shaky,5,2017-01-01,Daddy Yankee,Argentina,top200,MOVE_UP,153956.0,Shaky Shaky,...,2016-04-08,0.899,0.626,6.0,-4.228,0.0,0.2920,0.0631,0.873,4.0


In [5]:
# Obter as informações da base 

filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4277997 entries, 0 to 25276606
Data columns (total 23 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Unnamed: 0         int64         
 1   title              object        
 2   rank               int64         
 3   date               datetime64[ns]
 4   artist             object        
 5   region             object        
 6   chart              object        
 7   trend              object        
 8   streams            float64       
 9   album              object        
 10  popularity         float64       
 11  duration_ms        float64       
 12  explicit           float64       
 13  release_date       datetime64[ns]
 14  af_danceability    float64       
 15  af_energy          float64       
 16  af_key             float64       
 17  af_loudness        float64       
 18  af_mode            float64       
 19  af_speechiness     float64       
 20  af_liveness        float64  

In [6]:
#Exportar a base filtrada para a máquina 

filtered_df.to_csv("C:\\Users\\spotify_data.csv", index=False)

In [8]:

# Obter informações da base original

chunksize = 10**6  # Tamanho dos chunks para leitura
input_filepath = "C:\\Users\\rodri\\OneDrive\\Documentos\\Projeto ETL\\merged_data.csv" # Caminho da base original
output_filepath = "C:\\Users\\data_countries.csv"


# Nome da coluna de regiões e da coluna para filtrar
country_column = 'region'

data =[]

# Ler os dados em chunks e processar cada chunk
for chunk in pd.read_csv(input_filepath, chunksize=chunksize, low_memory=False):
    # Filtrar só as regiões 
    chunk=chunk['region']

    data.append(chunk)
    

data_countries = pd.concat(data)


In [9]:
# Contar as linhas 
data_countries_df = pd.DataFrame(data_countries)
data_countries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26174269 entries, 0 to 26174268
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   region  object
dtypes: object(1)
memory usage: 199.7+ MB


In [7]:
# Contar o número de países 

data_contries_list = data_countries_df.drop_duplicates(subset = 'region')
data_contries_list.info()


<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 0 to 14500298
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   region  70 non-null     object
dtypes: object(1)
memory usage: 1.1+ KB


In [2]:
# Transformar a tabela filtrada em DataFrame 

data_spotify_charts = pd.read_csv('C:\\Users\\rodri\\OneDrive\\Documentos\\Projeto ETL\\spotify_data.csv')

data_spotify_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277997 entries, 0 to 4277996
Data columns (total 23 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   title              object 
 2   rank               int64  
 3   date               object 
 4   artist             object 
 5   region             object 
 6   chart              object 
 7   trend              object 
 8   streams            float64
 9   album              object 
 10  popularity         float64
 11  duration_ms        float64
 12  explicit           float64
 13  release_date       object 
 14  af_danceability    float64
 15  af_energy          float64
 16  af_key             float64
 17  af_loudness        float64
 18  af_mode            float64
 19  af_speechiness     float64
 20  af_liveness        float64
 21  af_valence         float64
 22  af_time_signature  float64
dtypes: float64(13), int64(2), object(8)
memory usage: 750.7+ MB


2. Armazenar no banco de dados

In [13]:
# Conectar o banco de dados

from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import insert

usuario = 'root'
senha = '****'
host = 'localhost'
porta = '3306'
banco = 'spotify_charts'

engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{banco}')

In [14]:
# Exportar a base filtrada para o sql

chunk_size = 1000000

file_path = 'C:\\Users\\rodri\\OneDrive\\Documentos\\Projeto ETL\\spotify_data.csv'

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunk.to_sql('data_spotify_charts', con=engine, if_exists='append', index=False)
